# Score LCL dsQTL variants with Enformer

In this notebook, we score LCL dsQTLs with Enformer from the gReLU zoo.

In [1]:
# ! pip install git+https://github.com/Genentech/gReLU

In [1]:
import os
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import warnings

In [2]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

## Set experiment parameters

In [3]:
import grelu
grelu.__version__

'1.0.5.post1.dev13'

In [4]:
import grelu.lightning
import grelu.resources

/home/nairs51/miniforge3/envs/grelu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = grelu.resources.load_model(project="enformer",
                                   model_name="human",
                                   host="https://api.wandb.ai")

wandb: Currently logged in as: anony-mouse-444847806635717680 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Serializing object of type list that is 277336 bytes
wandb: WARNING Serializing object of type list that is 277336 bytes
wandb: WARNING Serializing object of type list that is 277336 bytes
wandb: WARNING Serializing object of type list that is 277336 bytes
wandb: WARNING Serializing object of type list that is 277336 bytes
wandb: WARNING Serializing object of type list that is 277336 bytes
wandb: Downloading large artifact human:latest, 941.03MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.7


List of Enformer tasks.

In [6]:
tasks = pd.DataFrame(model.data_params['tasks'])
tasks.head()

,name,file,clip,scale,sum_stat,description,assay,sample
0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...,DNASE,cerebellum male adult (27 years) and male adul...
1,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...,DNASE,frontal cortex male adult (27 years) and male ...
2,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion,DNASE,chorion
3,ENCFF463ZLQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:Ishikawa treated with 0.02% dimethyl sul...,DNASE,Ishikawa treated with 0.02% dimethyl sulfoxide...
4,ENCFF890OGQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM03348,DNASE,GM03348


In [7]:
tasks[tasks['name']=="ENCFF093VXI"]

,name,file,clip,scale,sum_stat,description,assay,sample
12,ENCFF093VXI,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM12878,DNASE,GM12878


In [8]:
TASK_IDX = 12

In [9]:
SEQLEN = model.data_params['train']['seq_len']
SEQLEN

196608

In [10]:
OUTLEN = model.data_params['train']['label_len']
OUTLEN

114688

In [11]:
BINSIZE = model.data_params['train']['bin_size']
BINSIZE

128

## Variants

In [12]:
# file can be downloaded from https://static-content.springer.com/esm/art%3A10.1038%2Fng.3331/MediaObjects/41588_2015_BFng3331_MOESM26_ESM.xlsx
variants_df = pd.read_csv("/home/nairs51/resources/LCL_dsQTL/41588_2015_BFng3331_MOESM26_ESM.tsv", sep='\t')
variants_df = variants_df[['chrom_hg19', 'pos_hg19', 'allele1', 'allele2', 'label', 'abs_gkm_SVM']]
variants_df = variants_df.set_axis(['chrom', 'pos', 'ref', 'alt', 'label', 'abs_gkm_SVM'], axis=1)
variants_df['label'] = variants_df['label'].replace(-1, 0)

variants_df['start'] = variants_df['pos']
variants_df['end'] = variants_df['start'] + 1

variants_df.head()

,chrom,pos,ref,alt,label,abs_gkm_SVM,start,end
0,chr1,856583,A,G,1,2.653531,856583,856584
1,chr1,911595,G,A,1,2.821422,911595,911596
2,chr1,1186502,T,A,1,7.167236,1186502,1186503
3,chr1,1227412,A,G,1,3.957382,1227412,1227413
4,chr1,1590575,A,G,1,4.785596,1590575,1590576


In [13]:
from grelu.data.preprocess import filter_blacklist, filter_chromosomes, filter_chrom_ends
from grelu.variant import filter_variants

In [14]:
variants = filter_variants(variants_df, max_del_len=0, max_insert_len=0, standard_bases=True)

Initial number of variants: 28309
Final number of variants: 28309


In [15]:
variants = filter_chromosomes(variants, include='autosomesXY')

Keeping 28309 intervals


Remove those near edge of chromosome, that will go out of bound for Enformer with variant at center.

In [16]:
variants = filter_chrom_ends(variants, genome='hg19', pad=SEQLEN//2)

Keeping 28274 intervals


## Predict

Aggregate the output in central 8 bins corresponding to 128*8=1024 bases.

In [17]:
OUTLEN//BINSIZE//2-4, OUTLEN//BINSIZE//2+4

(444, 452)

In [18]:
def logsum_length_aggfunc(x, axis, keepdims):
    return torch.log1p(torch.sum(x, axis=axis, keepdims=keepdims))

In [19]:
from grelu.transforms.prediction_transforms import Aggregate
lcl_score = Aggregate(tasks=[TASK_IDX], # ENCFF093VXI (GM12878 DNase)
                      positions=list(range(OUTLEN//BINSIZE//2-4, OUTLEN//BINSIZE//2+4)),
                      length_aggfunc=logsum_length_aggfunc, # output log sum
                      model=model)

In [20]:
import grelu.variant

odds = grelu.variant.predict_variant_effects(
    variants=variants,
    model=model, 
    seq_len=SEQLEN,
    devices=4, # Run on GPU 4
    num_workers=8,
    batch_size=4,
    genome="hg19",
    compare_func="subtract", 
    prediction_transform=lcl_score,
    return_ad=False, # Return an anndata object.
    rc = True, # Reverse complement the ref/alt predictions and average them.
)

making dataset


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/nairs51/miniforge3/envs/grelu/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting DataLoader 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28274/28274 [2:16:44<00:00,  3.45it/s]


In [22]:
odds.shape

(28274, 1, 1)

In [23]:
import sklearn.metrics

In [25]:
auprc = sklearn.metrics.average_precision_score(variants['label'], np.abs(odds.ravel()))
auprc

0.6018102676743534

In [27]:
pr, rec, _ = sklearn.metrics.precision_recall_curve(variants['label'], np.abs(odds.ravel()))

In [29]:
variants['enformer_task_12_ENCFF093VXI_LFC_rc'] = odds.ravel()
variants.head()

,chrom,pos,ref,alt,label,abs_gkm_SVM,start,end,enformer_task_12_ENCFF093VXI_LFC_rc
0,chr1,856583,A,G,1,2.653531,856583,856584,-0.200571
1,chr1,911595,G,A,1,2.821422,911595,911596,0.307417
2,chr1,1186502,T,A,1,7.167236,1186502,1186503,0.349024
3,chr1,1227412,A,G,1,3.957382,1227412,1227413,0.640315
4,chr1,1590575,A,G,1,4.785596,1590575,1590576,0.059268


In [31]:
variants.reset_index(drop=True).to_csv("./out/enformer_dsQTL.tsv", sep='\t', index=None)